In [1]:
# Set the Pyspark enviroment variables
import os
os.environ['SPARK_HOME'] = "/Users/patricia/Apps/Spark"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter' # only if you use jupyter notebook
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'notebook' # only if you use jupyter notebook
os.environ['PYSPARK_PYTHON'] = 'python'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc

In [4]:
# Create a SparkSession
spark = SparkSession.builder.appName("DataFrame-Demo").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/06 16:21:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

24/11/06 16:21:36 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


### Using RDDs

In [8]:
rdd = spark.sparkContext.textFile("./data/data.txt")
result_rdd = rdd.flatMap(lambda line: line.split(" ")) \
            .map(lambda word:(word,1))  \
            .reduceByKey(lambda a,b: a + b)  \
            .sortBy(lambda x:x[1],ascending=False)

In [9]:
result_rdd.take(10)

[('the', 12),
 ('of', 7),
 ('a', 7),
 ('distributed', 5),
 ('in', 5),
 ('Spark', 4),
 ('as', 3),
 ('is', 3),
 ('API', 3),
 ('on', 3)]

### Using DataFrames

In [10]:
df = spark.read.text("./data/data.txt")

result_df = df.selectExpr("explode(split(value, ' ')) as word").groupBy("word").count().orderBy(desc("count"))

In [12]:
result_df.take(10)

[Row(word='the', count=12),
 Row(word='of', count=7),
 Row(word='a', count=7),
 Row(word='in', count=5),
 Row(word='distributed', count=5),
 Row(word='Spark', count=4),
 Row(word='API', count=3),
 Row(word='RDD', count=3),
 Row(word='is', count=3),
 Row(word='on', count=3)]

In [13]:
spark.stop()